In [ ]:
# Parameters
CONTEXTLENGTH = 512  # will be overridden by papermill

this notebook loads tirex and makes predictions using it. Input context data is loaded from data/interim/data.csv. results are saved to data/processed/prediction_matrices/50/.

In [1]:
import os
os.environ['TIREX_NO_CUDA'] = '1'

import pandas as pd
import torch
import numpy as np
import logging
from pyprojroot import here

import sys
sys.path.append(str(here() / 'scripts'))
sys.path.append(str(here() / 'timesfm'))

from tirex import load_model, ForecastModel
from utils import (get_data,get_y_context, save)

/home/valtti/code/ed-ml-multivar/env-tirex/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
MAXMODELCONTEXT= 12840 # max context length for this model (no context length limit for this model)
MAXCONTEXT = 12840 # max context length for this dataset (2017-01-01 - 2018-06-19)
if CONTEXTLENGTH > MAXMODELCONTEXT:
    CONTEXTLENGTH = MAXMODELCONTEXT

if CONTEXTLENGTH > MAXCONTEXT:
    CONTEXTLENGTH = MAXCONTEXT

MODEL = 'tirex'
TARGET = 'occ'
FEATURESET = 'u'
TESTNAME = f'{TARGET}-{MODEL}-{FEATURESET}-{CONTEXTLENGTH}'
TESTSTART = pd.Timestamp('20180620')
TESTSIZE = 365*24
PREDICTIONLENGTH = 24

In [ ]:
logging.basicConfig(
        format='%(asctime)s - %(message)s', 
        level=logging.INFO,
        filename=here() / f"logs/logger/{TESTNAME}.log",      
        filemode='w'
        )
logging.info('Starting test.')
logging.info(f'MODEL: {MODEL} TARGET: {TARGET} FEATURESET:'
     f'{FEATURESET} CONTEXTLENGTH: {CONTEXTLENGTH}')

In [3]:
data = get_data(TARGET, MODEL)
cd = {
        'occ' : 'Target:Occupancy',
        'arr' : 'Target:Arrivals'
    }
target_name = cd[TARGET]
y = data[target_name]
y_context = get_y_context(data, TARGET, CONTEXTLENGTH, TESTSTART)

y_context = y_context.to_numpy()

In [4]:
model: ForecastModel = load_model("NX-AI/TiRex")

In [5]:
forecast = model.forecast(context=y_context, prediction_length=PREDICTIONLENGTH,
                          output_type='numpy')

In [13]:
mean_forecast = forecast[1]
save(model_name=MODEL, featureset_name=FEATURESET, target_name=TARGET,
context_length=CONTEXTLENGTH, test_start=TESTSTART, y_pred=mean_forecast)